# Time Series Forecasting for Energy Use

This project is selected to explore the use of basic time series forecasting techniques. The goal is not 100% accuracy but rather 95% for planning purposes. We will further refine the accuracy metrics.

First, we read in the datasets for energy usage and temperature data.

In [38]:
import pandas as pd
import datetime as dt

In [3]:
#reading in csv file, using the function parse_dates to autoconvert strings to 'datetime64[ns]'
temp_df = pd.read_csv('Milestone 1 Dataset\hr_temp_20170201-20200131_subset.csv',parse_dates = ['DATE'])

In [4]:
temp_df

,STATION,DATE,REPORT_TYPE,SOURCE,HourlyDryBulbTemperature
0,72520514762,2017-02-01 00:53:00,FM-15,7,37.0
1,72520514762,2017-02-01 01:53:00,FM-15,7,37.0
2,72520514762,2017-02-01 02:53:00,FM-15,7,36.0
3,72520514762,2017-02-01 03:53:00,FM-15,7,36.0
4,72520514762,2017-02-01 04:53:00,FM-15,7,36.0
...,...,...,...,...,...
26275,72520514762,2020-01-31 19:53:00,FM-15,7,34.0
26276,72520514762,2020-01-31 20:53:00,FM-15,7,33.0
26277,72520514762,2020-01-31 21:53:00,FM-15,7,33.0
26278,72520514762,2020-01-31 22:53:00,FM-15,7,33.0


In [5]:
temp_df.dtypes

STATION                              int64
DATE                        datetime64[ns]
REPORT_TYPE                         object
SOURCE                               int64
HourlyDryBulbTemperature           float64
dtype: object

In [6]:
#read in the megawatt usage data
load_df = pd.read_csv('Milestone 1 Dataset\hrl_load_metered - 20170201-20200131.csv ',parse_dates = ['datetime_beginning_ept'])
load_df

,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified
0,2/1/2017 5:00,2017-02-01 00:00:00,RFC,WEST,DUQ,DUQ,1419.881,True
1,2/1/2017 6:00,2017-02-01 01:00:00,RFC,WEST,DUQ,DUQ,1379.505,True
2,2/1/2017 7:00,2017-02-01 02:00:00,RFC,WEST,DUQ,DUQ,1366.106,True
3,2/1/2017 8:00,2017-02-01 03:00:00,RFC,WEST,DUQ,DUQ,1364.453,True
4,2/1/2017 9:00,2017-02-01 04:00:00,RFC,WEST,DUQ,DUQ,1391.265,True
...,...,...,...,...,...,...,...,...
26275,2/1/2020 0:00,2020-01-31 19:00:00,RFC,WEST,DUQ,DUQ,1618.484,True
26276,2/1/2020 1:00,2020-01-31 20:00:00,RFC,WEST,DUQ,DUQ,1580.925,True
26277,2/1/2020 2:00,2020-01-31 21:00:00,RFC,WEST,DUQ,DUQ,1545.354,True
26278,2/1/2020 3:00,2020-01-31 22:00:00,RFC,WEST,DUQ,DUQ,1478.832,True


In [7]:
load_df.dtypes

datetime_beginning_utc            object
datetime_beginning_ept    datetime64[ns]
nerc_region                       object
mkt_region                        object
zone                              object
load_area                         object
mw                               float64
is_verified                         bool
dtype: object

## 1. Load the energy and temperature datasets into a single pandas DataFrame that is matched by date.

In [10]:
#date in the temp_df seems to have been shifted by 53mins forward
#We can use offset to move this back, or just round down
#confirm this using pandas

temp_df['DATE'].dt.minute.unique()

array([53, 49, 48, 51, 50, 52], dtype=int64)

In [12]:
#since the mins are not uniformly shifted, we must rounddown.
#rounddown to have the same data across the two datasets

temp_df['DATE'] = temp_df['DATE'].dt.floor('h')

In [18]:
#perform a left join between wattage data and temp data
combined_df = load_df.merge (temp_df, how = 'left', left_on ='datetime_beginning_ept', right_on = 'DATE')
combined_df

,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified,STATION,DATE,REPORT_TYPE,SOURCE,HourlyDryBulbTemperature
0,2/1/2017 5:00,2017-02-01 00:00:00,RFC,WEST,DUQ,DUQ,1419.881,True,72520514762,2017-02-01 00:00:00,FM-15,7,37.0
1,2/1/2017 6:00,2017-02-01 01:00:00,RFC,WEST,DUQ,DUQ,1379.505,True,72520514762,2017-02-01 01:00:00,FM-15,7,37.0
2,2/1/2017 7:00,2017-02-01 02:00:00,RFC,WEST,DUQ,DUQ,1366.106,True,72520514762,2017-02-01 02:00:00,FM-15,7,36.0
3,2/1/2017 8:00,2017-02-01 03:00:00,RFC,WEST,DUQ,DUQ,1364.453,True,72520514762,2017-02-01 03:00:00,FM-15,7,36.0
4,2/1/2017 9:00,2017-02-01 04:00:00,RFC,WEST,DUQ,DUQ,1391.265,True,72520514762,2017-02-01 04:00:00,FM-15,7,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26275,2/1/2020 0:00,2020-01-31 19:00:00,RFC,WEST,DUQ,DUQ,1618.484,True,72520514762,2020-01-31 19:00:00,FM-15,7,34.0
26276,2/1/2020 1:00,2020-01-31 20:00:00,RFC,WEST,DUQ,DUQ,1580.925,True,72520514762,2020-01-31 20:00:00,FM-15,7,33.0
26277,2/1/2020 2:00,2020-01-31 21:00:00,RFC,WEST,DUQ,DUQ,1545.354,True,72520514762,2020-01-31 21:00:00,FM-15,7,33.0
26278,2/1/2020 3:00,2020-01-31 22:00:00,RFC,WEST,DUQ,DUQ,1478.832,True,72520514762,2020-01-31 22:00:00,FM-15,7,33.0


In [23]:
#keep only column i care about
combined_df = combined_df[['DATE','HourlyDryBulbTemperature','mw']]
combined_df

,DATE,HourlyDryBulbTemperature,mw
0,2017-02-01 00:00:00,37.0,1419.881
1,2017-02-01 01:00:00,37.0,1379.505
2,2017-02-01 02:00:00,36.0,1366.106
3,2017-02-01 03:00:00,36.0,1364.453
4,2017-02-01 04:00:00,36.0,1391.265
...,...,...,...
26275,2020-01-31 19:00:00,34.0,1618.484
26276,2020-01-31 20:00:00,33.0,1580.925
26277,2020-01-31 21:00:00,33.0,1545.354
26278,2020-01-31 22:00:00,33.0,1478.832


# 2 Type check

In [24]:
#we want DATE to be datetime64 b/c we want to create more date-related fields from them
combined_df.dtypes

DATE                        datetime64[ns]
HourlyDryBulbTemperature           float64
mw                                 float64
dtype: object

# 3. Create date-related variables for DEA and seasonality identification

In [29]:
combined_df['hour'] = combined_df['DATE'].dt.hour
combined_df['day_of_week'] = combined_df['DATE'].dt.dayofweek
combined_df['month'] = combined_df['DATE'].dt.month
combined_df['year'] = combined_df['DATE'].dt.year

In [30]:
combined_df

,DATE,HourlyDryBulbTemperature,mw,hour,day_of_week,month,year
0,2017-02-01 00:00:00,37.0,1419.881,0,2,2,2017
1,2017-02-01 01:00:00,37.0,1379.505,1,2,2,2017
2,2017-02-01 02:00:00,36.0,1366.106,2,2,2,2017
3,2017-02-01 03:00:00,36.0,1364.453,3,2,2,2017
4,2017-02-01 04:00:00,36.0,1391.265,4,2,2,2017
...,...,...,...,...,...,...,...
26275,2020-01-31 19:00:00,34.0,1618.484,19,4,1,2020
26276,2020-01-31 20:00:00,33.0,1580.925,20,4,1,2020
26277,2020-01-31 21:00:00,33.0,1545.354,21,4,1,2020
26278,2020-01-31 22:00:00,33.0,1478.832,22,4,1,2020


# 4. Set Index for Pandas DataFrame

In [31]:
combined_df.set_index('DATE')

,HourlyDryBulbTemperature,mw,hour,day_of_week,month,year
DATE,,,,,,
2017-02-01 00:00:00,37.0,1419.881,0,2,2,2017
2017-02-01 01:00:00,37.0,1379.505,1,2,2,2017
2017-02-01 02:00:00,36.0,1366.106,2,2,2,2017
2017-02-01 03:00:00,36.0,1364.453,3,2,2,2017
2017-02-01 04:00:00,36.0,1391.265,4,2,2,2017
...,...,...,...,...,...,...
2020-01-31 19:00:00,34.0,1618.484,19,4,1,2020
2020-01-31 20:00:00,33.0,1580.925,20,4,1,2020
2020-01-31 21:00:00,33.0,1545.354,21,4,1,2020


In [32]:
#check null values
combined_df.isnull().sum() #we see clear issues with temperature data, there are missing values

DATE                         0
HourlyDryBulbTemperature    37
mw                           0
hour                         0
day_of_week                  0
month                        0
year                         0
dtype: int64

In [37]:
combined_df['HourlyDryBulbTemperature'] = combined_df['HourlyDryBulbTemperature'].interpolate(method = 'linear')

# 6. Splitting the dataset into train and test sets

In [39]:
train_df = combined_df[combined_df['DATE'] < dt.datetime(2020,1,1)]

In [40]:
test_df = combined_df[combined_df['DATE'] >= dt.datetime(2020,1,1)]

In [41]:
#checking the bottom records to make sure
train_df.tail(5)

,DATE,HourlyDryBulbTemperature,mw,hour,day_of_week,month,year
25531,2019-12-31 19:00:00,32.0,1606.134,19,1,12,2019
25532,2019-12-31 20:00:00,32.0,1565.014,20,1,12,2019
25533,2019-12-31 21:00:00,32.0,1515.905,21,1,12,2019
25534,2019-12-31 22:00:00,30.0,1474.519,22,1,12,2019
25535,2019-12-31 23:00:00,30.0,1421.717,23,1,12,2019


In [42]:
test_df.tail(5)

,DATE,HourlyDryBulbTemperature,mw,hour,day_of_week,month,year
26275,2020-01-31 19:00:00,34.0,1618.484,19,4,1,2020
26276,2020-01-31 20:00:00,33.0,1580.925,20,4,1,2020
26277,2020-01-31 21:00:00,33.0,1545.354,21,4,1,2020
26278,2020-01-31 22:00:00,33.0,1478.832,22,4,1,2020
26279,2020-01-31 23:00:00,34.0,1405.188,23,4,1,2020
